<font size='6' color='red'>PREMODELING FUNCTIONS</font>

## For Project Deliverable

In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_colwidth',100,'display.max_columns',999)
pd.set_option('mode.chained_assignment',None)

import pickle

from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
%matplotlib inline

import warnings

In [2]:
raw_data = pd.read_csv('raw_data.csv')
raw_data

,beds,baths,sqft,year_built,lot_size,property_type,exterior_walls,roof,basement,restaurants,groceries,nightlife,cafes,shopping,arts_entertainment,beauty_spas,active_life,median_age,married,college_grad,property_tax,insurance,median_school,num_schools,tx_year
0,1,1,584,2013,0,Apartment / Condo / Townhouse,Wood Siding,NaN,0.0,107,9,30,19,89,6,47,58,33.0,65.0,84.0,234.0,81.0,9.0,3.0,2013
1,1,1,612,1965,0,Apartment / Condo / Townhouse,Brick,Composition Shingle,1.0,105,15,6,13,87,2,26,14,39.0,73.0,69.0,169.0,51.0,3.0,3.0,2006
2,1,1,615,1963,0,Apartment / Condo / Townhouse,Wood Siding,NaN,0.0,183,13,31,30,101,10,74,62,28.0,15.0,86.0,216.0,74.0,8.0,3.0,2012
3,1,1,618,2000,33541,Apartment / Condo / Townhouse,Wood Siding,NaN,0.0,198,9,38,25,127,11,72,83,36.0,25.0,91.0,265.0,92.0,9.0,3.0,2005
4,1,1,634,1992,0,Apartment / Condo / Townhouse,Brick,NaN,0.0,149,7,22,20,83,10,50,73,37.0,20.0,75.0,88.0,30.0,9.0,3.0,2002
5,1,1,641,1947,0,Apartment / Condo / Townhouse,Brick,NaN,0.0,146,10,23,27,86,9,60,52,28.0,15.0,86.0,168.0,58.0,8.0,3.0,2004
6,1,1,642,1944,0,Single-Family,Brick,NaN,0.0,159,13,36,17,92,12,66,50,28.0,36.0,88.0,176.0,61.0,7.0,3.0,2011
7,1,1,650,2000,33541,Apartment / Condo / Townhouse,Wood Siding,NaN,0.0,198,9,38,25,127,11,72,83,36.0,25.0,91.0,266.0,92.0,9.0,3.0,2005
8,1,1,660,1983,0,Apartment / Condo / Townhouse,Brick,NaN,0.0,51,8,6,2,40,18,32,41,36.0,49.0,77.0,188.0,65.0,6.0,3.0,2013
9,1,1,664,1965,0,Apartment / Condo / Townhouse,Brick,NaN,0.0,119,10,26,25,183,13,70,36,57.0,13.0,83.0,147.0,51.0,6.0,3.0,2007


In [3]:
def clean_data(df):
    df.drop_duplicates()
    df['basement'] = df.basement.fillna(0)
    df.roof.replace('composition','Composition',inplace=True)
    df.roof.replace('asphalt','Asphalt',inplace=True)
    df.roof.replace(['shake-shingle','asphalt,shake-shingle'],'Shake Shingle',inplace=True)
    df.exterior_walls.replace('Rock, Stone','Masonry',inplace=True)
    df.exterior_walls.replace(['Concrete','Block'],'Concrete Block',inplace=True)
    df = df[df.lot_size<=500000]
    for column in df.select_dtypes(include=['object']):
        df[column] = df[column].fillna('Missing')
    return df 

In [4]:
cleaned_data = clean_data(raw_data)
cleaned_data.head()

,beds,baths,sqft,year_built,lot_size,property_type,exterior_walls,roof,basement,restaurants,groceries,nightlife,cafes,shopping,arts_entertainment,beauty_spas,active_life,median_age,married,college_grad,property_tax,insurance,median_school,num_schools,tx_year
0,1,1,584,2013,0,Apartment / Condo / Townhouse,Wood Siding,Missing,0.0,107,9,30,19,89,6,47,58,33.0,65.0,84.0,234.0,81.0,9.0,3.0,2013
1,1,1,612,1965,0,Apartment / Condo / Townhouse,Brick,Composition Shingle,1.0,105,15,6,13,87,2,26,14,39.0,73.0,69.0,169.0,51.0,3.0,3.0,2006
2,1,1,615,1963,0,Apartment / Condo / Townhouse,Wood Siding,Missing,0.0,183,13,31,30,101,10,74,62,28.0,15.0,86.0,216.0,74.0,8.0,3.0,2012
3,1,1,618,2000,33541,Apartment / Condo / Townhouse,Wood Siding,Missing,0.0,198,9,38,25,127,11,72,83,36.0,25.0,91.0,265.0,92.0,9.0,3.0,2005
4,1,1,634,1992,0,Apartment / Condo / Townhouse,Brick,Missing,0.0,149,7,22,20,83,10,50,73,37.0,20.0,75.0,88.0,30.0,9.0,3.0,2002


In [5]:
def engineer_features(df):
    df['two_two'] = ((df.beds==2)& (df.baths==2)).astype(int)
    df['during_recession'] = df.tx_year.between(2010,2013).astype(int)
    df['property_age'] = df.tx_year - df.year_built
    df = df[df.property_age >=0]
    df['school_score'] = df.num_schools * df.median_school
    df.exterior_walls.replace(['Wood Siding','Wood Shingle','Wood'],'Wood',inplace=True)
    other_exterior_walls=['Stucco','Other','Asbestos shingle','Concrete Block','Masonry']
    df.exterior_walls.replace(other_exterior_walls,'Other',inplace=True)
    df.roof.replace(['Composition','Wood Shake/ Shingles'],'Composition Shingle',inplace=True)
    other_roofs=['Other','Gravel/Rock','Roll Composition','Slate','Built-up','Asbestos','Metal']
    df.roof.replace(other_roofs,'Other',inplace=True)
    df = pd.get_dummies(df,columns=['exterior_walls','roof','property_type'])
    df = df.drop(['tx_year','year_built'],axis=1)
    return df

In [6]:
augmented_data = engineer_features(cleaned_data)
augmented_data.head()

,beds,baths,sqft,lot_size,basement,restaurants,groceries,nightlife,cafes,shopping,arts_entertainment,beauty_spas,active_life,median_age,married,college_grad,property_tax,insurance,median_school,num_schools,two_two,during_recession,property_age,school_score,exterior_walls_Brick,exterior_walls_Brick veneer,exterior_walls_Combination,exterior_walls_Metal,exterior_walls_Missing,exterior_walls_Other,exterior_walls_Siding (Alum/Vinyl),exterior_walls_Wood,roof_Asphalt,roof_Composition Shingle,roof_Missing,roof_Other,roof_Shake Shingle,property_type_Apartment / Condo / Townhouse,property_type_Single-Family
0,1,1,584,0,0.0,107,9,30,19,89,6,47,58,33.0,65.0,84.0,234.0,81.0,9.0,3.0,0,1,0,27.0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0
1,1,1,612,0,1.0,105,15,6,13,87,2,26,14,39.0,73.0,69.0,169.0,51.0,3.0,3.0,0,0,41,9.0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
2,1,1,615,0,0.0,183,13,31,30,101,10,74,62,28.0,15.0,86.0,216.0,74.0,8.0,3.0,0,1,49,24.0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0
3,1,1,618,33541,0.0,198,9,38,25,127,11,72,83,36.0,25.0,91.0,265.0,92.0,9.0,3.0,0,0,5,27.0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0
4,1,1,634,0,0.0,149,7,22,20,83,10,50,73,37.0,20.0,75.0,88.0,30.0,9.0,3.0,0,0,10,27.0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0


In [7]:
with open('final_model.pkl','rb') as f:
    model = pickle.load(f)

In [8]:
model

Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
           oob_score=False, random_state=123, verbose=0, warm_start=False))])

In [9]:
with warnings.catch_warnings():          # to ignore DataConversionWarning due to StandardScaler
    warnings.simplefilter("ignore")
    pred = model.predict(augmented_data)
pred

array([304540.19 , 234675.5  , 280900.885, ..., 666033.34 , 721630.71 ,
       701576.225])

## CONSTRUCTING A MODEL CLASS FINALLY

In [10]:
class RealEstateModel:
    
    def __init__(self, model_location):
        with open(model_location, 'rb') as f:
            self.model = pickle.load(f)
            
    def predict(self, X_new, clean=True, augment=True):
        if clean:
            X_new = self.clean_data(X_new)
            
        if augment:
            X_new = self.engineer_features(X_new)
            
        return X_new, self.model.predict(X_new)
    
    def clean_data(self, df):
        df.drop_duplicates()
        df['basement'] = df.basement.fillna(0)
        df.roof.replace('composition','Composition',inplace=True)
        df.roof.replace('asphalt','Asphalt',inplace=True)
        df.roof.replace(['shake-shingle','asphalt,shake-shingle'],'Shake Shingle',inplace=True)
        df.exterior_walls.replace('Rock, Stone','Masonry',inplace=True)
        df.exterior_walls.replace(['Concrete','Block'],'Concrete Block',inplace=True)
        df = df[df.lot_size<=500000]
        for column in df.select_dtypes(include=['object']):
            df[column] = df[column].fillna('Missing')
        return df
    
    def engineer_features(self, df):
        df['two_two'] = ((df.beds==2)& (df.baths==2)).astype(int)
        df['during_recession'] = df.tx_year.between(2010,2013).astype(int)
        df['property_age'] = df.tx_year - df.year_built
        df = df[df.property_age >=0]
        df['school_score'] = df.num_schools * df.median_school
        df.exterior_walls.replace(['Wood Siding','Wood Shingle','Wood'],'Wood',inplace=True)
        other_exterior_walls=['Stucco','Other','Asbestos shingle','Concrete Block','Masonry']
        df.exterior_walls.replace(other_exterior_walls,'Other',inplace=True)
        df.roof.replace(['Composition','Wood Shake/ Shingles'],'Composition Shingle',inplace=True)
        other_roofs=['Other','Gravel/Rock','Roll Composition','Slate','Built-up','Asbestos','Metal']
        df.roof.replace(other_roofs,'Other',inplace=True)
        df = pd.get_dummies(df,columns=['exterior_walls','roof','property_type'])
        df = df.drop(['tx_year','year_built'],axis=1)
        return df

##### Checking if model class is working properly by initializing an instance

In [11]:
estate_model = RealEstateModel('final_model.pkl')

In [12]:
with warnings.catch_warnings():          # to ignore DataConversionWarning due to StandardScaler
    warnings.simplefilter("ignore")
    _, pred1 = estate_model.predict(raw_data, clean=True, augment=True)

    _, pred2 = estate_model.predict(cleaned_data, clean=False, augment=True)

    _, pred3 = estate_model.predict(augmented_data, clean=False, augment=False)

In [13]:
pred1

array([304540.19 , 234675.5  , 280900.885, ..., 666033.34 , 721630.71 ,
       701576.225])

In [14]:
np.array_equal(pred1, pred2) & np.array_equal(pred2, pred3)

True

In [15]:
type(pred1)

numpy.ndarray

<font size='4' color='blue'>Run executable file real_estate.py on command line with arguments- data location, output location & model location</font>

In [16]:
# after running the real_estate.py file in the command line with arguments the prediction.csv will be generated
pd.read_csv('prediction.csv')

,beds,baths,sqft,lot_size,basement,restaurants,groceries,nightlife,cafes,shopping,arts_entertainment,beauty_spas,active_life,median_age,married,college_grad,property_tax,insurance,median_school,num_schools,two_two,during_recession,property_age,school_score,exterior_walls_Brick,exterior_walls_Brick veneer,exterior_walls_Combination,exterior_walls_Metal,exterior_walls_Missing,exterior_walls_Other,exterior_walls_Siding (Alum/Vinyl),exterior_walls_Wood,roof_Asphalt,roof_Composition Shingle,roof_Missing,roof_Other,roof_Shake Shingle,property_type_Apartment / Condo / Townhouse,property_type_Single-Family,predicted tx_price
0,1,1,584,0,0.0,107,9,30,19,89,6,47,58,33.0,65.0,84.0,234.0,81.0,9.0,3.0,0,1,0,27.0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,304540.190
1,1,1,612,0,1.0,105,15,6,13,87,2,26,14,39.0,73.0,69.0,169.0,51.0,3.0,3.0,0,0,41,9.0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,234675.500
2,1,1,615,0,0.0,183,13,31,30,101,10,74,62,28.0,15.0,86.0,216.0,74.0,8.0,3.0,0,1,49,24.0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,280900.885
3,1,1,618,33541,0.0,198,9,38,25,127,11,72,83,36.0,25.0,91.0,265.0,92.0,9.0,3.0,0,0,5,27.0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,379925.125
4,1,1,634,0,0.0,149,7,22,20,83,10,50,73,37.0,20.0,75.0,88.0,30.0,9.0,3.0,0,0,10,27.0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,317349.370
5,1,1,641,0,0.0,146,10,23,27,86,9,60,52,28.0,15.0,86.0,168.0,58.0,8.0,3.0,0,0,57,24.0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,261624.000
6,1,1,642,0,0.0,159,13,36,17,92,12,66,50,28.0,36.0,88.0,176.0,61.0,7.0,3.0,0,1,67,21.0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,247227.210
7,1,1,650,33541,0.0,198,9,38,25,127,11,72,83,36.0,25.0,91.0,266.0,92.0,9.0,3.0,0,0,5,27.0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,383220.875
8,1,1,660,0,0.0,51,8,6,2,40,18,32,41,36.0,49.0,77.0,188.0,65.0,6.0,3.0,0,1,30,18.0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,244468.260
9,1,1,664,0,0.0,119,10,26,25,183,13,70,36,57.0,13.0,83.0,147.0,51.0,6.0,3.0,0,0,42,18.0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,248284.500
